# Using Monte Carlo Tree Search for your Fantasy Football draft
Two more months till the next American Football season kicks off, which means Fantasy Football players around the world are preparing for their upcoming league draft. In this blog we will use the Monte Carlo Tree Search algorithm to optimize our next pick in a typical snake draft. If you are not familiar with Fantasy Football or snake drafts, you can find a good introduction at ESPN's Fantasy Football 101. We will look at a standard PPR scoring league on ESPN consisting of ten teams, but these settings should be easy to change to fit your own league's.

In [13]:
class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.turns = turns
        self.freeagents = freeagents
        self.playerJustMoved = playerjm

In [ ]:
import pandas as pd

In [10]:
nfl_players = pd.read_csv("nfl_players.csv", index_col=0)

In [11]:
nfl_players

,name,team,position,points
espn_id,,,,
15825,Le'Veon Bell,PIT,RB,351.1
17683,Todd Gurley II,LAR,RB,325.8
17442,David Johnson,ARI,RB,322.8
13934,Antonio Brown,PIT,WR,320.3
2330,Tom Brady,NE,QB,316.2
8439,Aaron Rodgers,GB,QB,313.3
13994,Cam Newton,CAR,QB,303.9
17795,Carson Wentz,PHI,QB,298.3
14881,Russell Wilson,SEA,QB,296.5


In [21]:
a = nfl_players.iloc[1]
[a, a]

[name        Todd Gurley II
 team                   LAR
 position                RB
 points               325.8
 Name: 17683, dtype: object, name        Todd Gurley II
 team                   LAR
 position                RB
 points               325.8
 Name: 17683, dtype: object]

In [ ]:
def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    WEIGHTS_POS = {
        ["QB"]: [.6, .4],
        ["WR"]: [.7, .7, .4, .2],
        ["RB"]: [.7, .7, .4, .2],
        ["TE"]: [.6, .4],
        ["RB", "WR", "TE"]: [.6, .4],
        ["D"]: [.6, .3, .1],
        ["K"]: [.5, .2, .2, .1]
    }

    if playerjm is None: return 0

    roster = self.rosters[playerjm].sort_values("points", ascending=False)
    result = 0
    for (pos, pts) in zip(roster["position"], roster["points"]):
        
        
        res = 0
        for (pos, w) in WEIGHTS_POS:
            p = next((p for p in roster if set(p.positions).intersection(pos)), None)
            if p:
                points = p.points
                roster.remove(p)
            else:
                ps = [p.points for p in self.freeagents if
                        set(p.positions).intersection(pos)]
                if len(ps) > 3: ps = ps[:3]
                points = float(sum(ps)) / max(len(ps), 1)
            res += points * w
        return res

In [18]:
a

In [20]:
nfl_players.iloc[1]

name        Todd Gurley II
team                   LAR
position                RB
points               325.8
Name: 17683, dtype: object

In [22]:
type(a)

pandas.core.series.Series